In [ ]:
try:
    # settings colab:
    import google.colab
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Series de Tiempo - Práctica Guiada

<a id="section_toc"></a> 
## Tabla de Contenidos

[1- Intro](#section_intro)

[2- Descomposición de una serie de tiempo](#section_descomposicion)

$\hspace{.5cm}$[2.1- Tendencia](#section_tendencia)

$\hspace{.5cm}$[2.2- Estacionalidad](#section_estacionalidad)

[3- Primeros modelos para prónosticos](#section_modelos_basicos)

$\hspace{.5cm}$[3.1- Análisis exploratorio y preprocesamiento de los datos](#section_eda_preprocesamiento)

$\hspace{.5cm}$[3.2- Media Constante](#section_media_constante)

$\hspace{.5cm}$[3.3- Random Walk](#section_random_walk)

$\hspace{.5cm}$[3.4- Tendencia Lineal](#section_tendencia_lineal)

$\hspace{.5cm}$[3.5- Tendencia Cuadrática](#section_tendencia_cuadratica)

$\hspace{.5cm}$[3.6- Tendencia Transformación Logarítmica](#section_tendencia_transformacion_log)

$\hspace{.5cm}$[3.7- Tendencia Transformación Logarítmica + Estacionalidad](#section_tendencia_transformacion_log_estacionalidad)

$\hspace{.5cm}$[3.8- Single Exponential Smoothing](#section_exponential_smoothing)

[4- Modelos de ciclos para series de tiempo](#section_modelos_ciclos)

$\hspace{.5cm}$[4.1- Series Estacionarias](#section_series_estacionarias)

$\hspace{.5cm}$[4.2- Modelo ARIMA](#section_modelo_arima)

---

<a id="section_intro"></a> 
###  1- Intro
[volver a TOC](#section_toc)

Esta clase sigue principalmente la exposición de *Elements of Forecasting (2001)* de Francis X. Diebold. 

#### ¿Qué es una serie de tiempo?

Podemos definir una **serie de tiempo** como un conjunto de observaciones tomadas en **intervalos regulares** que se encuentran **ordenadas** por el momento en que se produjeron.

El análisis de series de tiempo se suele utilizar para proyectar o pronosticar la evolución de una variable a lo largo del tiempo, a partir de información previa sobre esa misma variable. Es decir, tenemos que proyectar una variable determinada (eje y) en función del tiempo (eje x). 

Un ejemplo de serie de tiempo puede ser la evolución del precio de un determinado activo financiero a lo largo del tiempo como en la siguiente imagen:

![image.png](img/Picture1.png)

Otros ejemplos pueden ser:

- Variables macroeconómicas, como el PBI, inflación, reservas del BCRA, etc.
- Ventas de comercios
- Consumo energético


<div id="caja1" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>¿Cuáles podrían ser otros ejemplos de series de tiempo? <br/> 

<a id="section_descomposicion"></a> 
### 2- Descomposición de una serie de tiempo
[volver a TOC](#section_toc)

Existen distintos tipos de modelos para tratar las series de tiempo. 
Una opción es modelar una serie de tiempo como compuesta por **cuatro componentes**:
- **Tendencia**: componente “permanente”, el efecto persiste en el largo plazo. Se puede interpretar como lo que motiva el cambio a largo plazo de la media.
- **Estacionalidad**: esta componente aporta movimientos periódicos a la serie.
- **Ciclos**: se entiende por cualquier tipo de dinámica no capturada por la tendencia o la estacionalidad. Estamos ante la presencia de ciclos cuando observamos algún tipo de dinámica mediante la cual el presente está vinculado al pasado. No tiene por qué ser un ciclo rígido.
- **Componente aleatoria**: son shocks que no presentan un efecto duradero, ya que las suponemos i.i.d. con media 0 y varianza constante.

De manera aditiva, podemos decir entonces que: 

$$ y_t = T_t + S_t + C_t + \varepsilon_t. $$

… donde T es la tendencia, S es el efecto estacional C es el ciclo y $ \varepsilon $ es el error aleatorio.

<a id="section_tendencia"></a> 
#### 2.1- Tendencia
[volver a TOC](#section_toc)

La **tendencia** corresponde a la **evolución de largo plazo** de la variable que estamos analizando. Puede ser generada por la lenta evolución y cambios en las preferencias, la tecnología, las instituciones o en la demografía.


###### Ejemplo 1: Tasa de participación de las mujeres en la fuerza laboral (USA)
<img src="img/Picture2.png" width="450">


Podemos modelar la tendencia de diferentes maneras, dependiendo de la dinámica de la tendencia. 

##### Tendencia lineal:

$$ T_t = \beta_0 + \beta_1 TIME_t + \varepsilon_t. $$

Donde la variable *TIME* es una variable que construimos artificialmente, llamada “dummy de tiempo”.

Veamos unos ejemplos de tendencia lineal al variar los parámetros:

<img src="img/Picture3.png" width="450">

Veamos ahora cómo ajusta la tendencia lineal a la serie de la participación de las mujeres en la fuerza laboral de USA:
<img src="img/Picture4.png" width="450">
Vemos que la tendencia lineal ajusta bien a la tendencia de nuestra serie. Vemos que en el resíduo no parecen quedar dinámicas relacionadas a la tendencia.

<br/> 
 
###### Ejemplo 2: Volumen del New York Exchange
<img src="img/Picture5.png" width="450">


Si la tendencia que estamos buscando modelar presenta una dinámica no lineal, podemos incluir un **término cuadrático** al modelo. Lo obtenemos simplemente elevando al cuadrado a la dummy de tiempo:

$$ T_t = \beta_0 + \beta_1 TIME_t + \beta_2 TIME^{2}_t +\varepsilon_t. $$

Veamos qué tal ajusta nuestra tendencia cuadrática a la serie del NY Exchange:

<img src="img/Picture6.png" width="450">
Vemos que la tendencia cuadrática en este caso no ajusta bien a la tendencia de nuestra serie. Podemos observarlo en el residuo de nuestro modelo.

Probemos modelar a la serie del NY Exchange de otra manera. Si la tendencia está caracterizada por una tasa de crecimiento constante, podemos usar un **modelo exponencial**:

$$ T_t = \beta_0 e^{\beta_1 TIME_t}\varepsilon_t. $$

Si en lugar de modelar directamente la tendencia, modelamos su **logaritmo natural**, volvemos a tener un modelo lineal:

$$ \ln{T_t} = \ln{\beta_0} + \beta_1 TIME_t + \ln{\varepsilon_t}. $$

<img src="img/Picture7.png" width="450">
Vemos que el modelo logarítmico sí ajusta correctamente a nuestra serie.

Es relevante también tener en cuenta que realizar la transformación logarítmica también es útil cuando queremos estabilizar la dispersión de nuestra serie. 

Tenemos que tener en cuenta que debemos **revertir la transformación (back-transform)** para obtener forecasts en la escala original.


<a id="section_estacionalidad"></a> 
#### 2.2- Estacionalidad
[volver a TOC](#section_toc)

Un **patrón estacional** se repite con regularidad. La estacionalidad surge de vínculos entre el calendario y las tecnologías, las preferencias o las instituciones.

Por ejemplo podemos ver un patrón estacional en la venta de bebidas alcohólicas en Estados Unidos:

<img src="img/Picture8.png" width="450">

Podemos modelar la estacionalidad utilizando variables dummy. En el siguiente ejemplo modelamos una estacionalidad trimestral:

<img src="img/Picture9.png" width="300">

$$ S_t = \sum_{i=1}^{s} \gamma_iD_it  + \varepsilon_t. $$

Podemos agregar otro tipo de dummies, por ejemplo por días de vacaciones/feriados o ajustes por cantidad de días de comercio, etc.

<div id="caja1" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>¿Se les ocurren otros fenómenos que podrían generar patrones estacionales? ¿Cómo los modelarían?<br/> 

<a id="section_modelos_basicos"></a> 
### 3- Modelos básicos para pronósticos
[volver a TOC](#section_toc)

Pasemos al código para poner estos conceptos en práctica. 

En esta clase vamos a usar series de tiempo para predecir las ventas de medicamentos para diabetes en Australia.


***Nota: en esta práctica guiada, vas a encontrar algunas celdas donde vas a tener que completar con tu propio código. Son pequeños ejercicios simples, para usar código que ya deberías conocer de clases anteriores o que ya se presentó en esta práctica. Completarlo te va a servir para fijar mejor los conceptos y corroborar tu comprensión del tema.***

Primero importamos las librerías necesarias para trabajar con datos y visualizar:

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

from scipy import stats
from statistics import mode

import warnings
warnings.filterwarnings('ignore')

<a id="section_eda_preprocesamiento"></a> 
#### 3.1- Análisis exploratorio y preprocesamiento de los datos:
[volver a TOC](#section_toc)

Creamos nuestro DataFrame y realizamos un ploteo inicial:

In [ ]:
df = pd.read_csv('../Data/a10.csv', parse_dates=['date'], index_col='date')

# Función que plotea la serie:
def plot_df(df, x, y, title="", xlabel='Fecha', ylabel='Valor', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

plot_df(df, x=df.index, y=df.value,\
        title='Ventas mensuales de medicamentos para diabetes en Australia, 1992 a 2008')

Exploramos las primeras 5 observaciones de nuestro df:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df.head()

Ploteamos la estacionalidad:

In [ ]:
# Preparamos los datos:
df['year'] = [d.year for d in df.index]
df['month'] = [d.strftime('%b') for d in df.index]
years = df['year'].unique()

# Preparamos los colores:
np.random.seed(100)
mycolors = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(years), replace=False)

# Ploteamos
plt.figure(figsize=(16,12), dpi= 80)
for i, y in enumerate(years):
    if i > 0:        
        plt.plot('month', 'value', data=df.loc[df.year==y, :], color=mycolors[i], label=y)
        plt.text(df.loc[df.year==y, :].shape[0]-.9, df.loc[df.year==y, 'value'][-1:].values[0], y, fontsize=12, color=mycolors[i])


plt.gca().set(xlim=(-0.3, 11), ylim=(2, 30), ylabel='$Ventas$', xlabel='$Mes$')
plt.yticks(fontsize=12, alpha=.7)
plt.title("Gráfico Estacional de ventas de medicamentos", fontsize=20);

In [ ]:
# Definimos los plots:
fig, axes = plt.subplots(1, 2, figsize=(20,7), dpi= 80)
sns.boxplot(x='year', y='value', data=df, ax=axes[0])
sns.boxplot(x='month', y='value', data=df.loc[~df.year.isin([1991, 2008]), :])

# Seteamos los títulos:
axes[0].set_title('Box Plot Anual\n(Tendencia)', fontsize=18); 
axes[1].set_title('Box Plot Mensual\n(Estacionalidad)', fontsize=18);

Veamos el shape de nuestro df:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df.shape

Vamos a crear una **dummy de tiempo**, es decir indicador numérico para el período de tiempo,  para poder modelar la tendencia lineal.

También vamos a crear una variable que sea la dummy de tiempo elevada al cuadrado para modelar la tendencia cuadrática:

In [ ]:
df["timeIndex"] = pd.Series(np.arange(len(df['value'])), index=df.index)
df["timeIndex_sq"] = df["timeIndex"]**2

In [ ]:
# Creamos variables dummy para los meses:
dummies_mes = pd.get_dummies(df['month'], drop_first=True)
dummies_mes.head()

In [ ]:
# Hacemos el join entre el DataFrame con la serie de tiempo y las dummies:

# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df = df.join(dummies_mes)
df.sample(10)

Vamos a hacer un split entre train y test, teniendo en cuenta que, al tratarse de una serie de tiempo, tenemos que poner *shuffle=False*.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=12, random_state=42, shuffle=False)

<div id="caja1" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>¿Qué problema metodológico generaría hacer el split con shuffle=True? <br/> 

Verificamos la continuidad entre los sets de entrenamiento y de testeo:

In [ ]:
# Vemos los últimos datos del set de entrenamiento:

df_train.tail()

In [ ]:
# Vemos los primeros datos del set de testeo:

# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.head()

Vemos que hay continuidad entre el set de entrenamiento y el de testeo. Podemos entrenar los modelos con datos hasta septiembre de 2006 y testear con datos desde octubre en adelante, sin riesgo de data leakage. 

Ahora sí, pasemos a definir nuestros primeros modelos básicos de series de tiempo:

<a id="section_media_constante"></a> 
#### 3.2- Media Constante:
[volver a TOC](#section_toc)

La **media constante** es el modelo más básico e ingenuo de todos. Consiste simplemente en tomar la media del set de entrenamiento y utilizar ese valor para realizar predicciones. Puede usarse como un baseline para comparar otros modelos, pero difícilmente sea nuestro modelo final.

In [ ]:
# Calculamos el promedio:
model_mean_pred = df_train['value'].mean()

# La predicción es fija y es la misma para el set de testeo y de entrenamiento:
df_train["Mean"] = model_mean_pred
df_test["Mean"] = model_mean_pred

<div id="caja1" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>¿Tendría sentido calcular la media para el set de testeo y estimar la bondad del modelo con ese valor?¿Por qué? <br/>

In [ ]:
# Ploteamos los valores del set de entrenamiento y el modelo:
df_train.plot(kind="line", y = ["value", "Mean"]);

In [ ]:
# Ploteamos los valores del set de testeo y el modelo

# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind="line", y = ["value", "Mean"]);

* ¿Cuán bien ajusta esta estimación?

Primero definimos una función que calcula el **RMSE**. 

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

In [ ]:
model_mean_RMSE = RMSE(df_test.Mean, df_test.value)
model_mean_RMSE

In [ ]:
# Guardamos el resultado en un DataFrame:
df_Results = pd.DataFrame(columns = ["Model", "RMSE"])
df_Results.loc[0, "Model"] = "Mean"
df_Results.loc[0, "RMSE"] = model_mean_RMSE
df_Results.head()

<a id="section_random_walk"></a> 
#### 3.3- Random Walk:
[volver a TOC](#section_toc)

Decimos que un proceso $ {Y_t} $ es **random walk** (sigue una trayectoria al azar si):

$$ \ {Y_t} = Y_ {t-1} + \varepsilon,  $$

siendo epsilon **ruido blanco**. Si al modelo anterior le añadimos una constante $d$, obtenemos un **random walk con deriva (with drift)**:

$$ \ {Y_t} = Y_ {t-1} + d + \varepsilon $$

Cuando nos enfrentamos a una serie de tiempo que muestra una evolución irregular, la mejor estrategia no es tratar de predecir directamente el nivel de la serie en cada período (es decir, el valor Yt), sino el cambio que ocurre de un período al siguiente (es decir, la diferencia Yt - Yt-1). **Se observa la primera diferencia (o lag) de la serie para encontrar un patrón predecible**.

A los efectos del pronóstico del próximo período, puede ser tan bueno predecir el nivel como la variación, ya que el cambio predicho puede agregarse al nivel actual para generar un nivel pronosticado. El caso más simple de dicho modelo es uno que siempre predice que el siguiente cambio será cero, como si la serie tuviera la misma probabilidad de subir o bajar en el próximo período, independientemente de lo que haya sucedido en el pasado.

Generamos el desplazamiento de nuestro nuestro target en el set de entrenamiento (lag=1):

In [ ]:
df_train["valueShift1"] = df_train.value.shift()

# La primera observación nos va a quedar en nan, la reemplazamos por el valor siguente:
df_train["valueShift1"].fillna(method='bfill', inplace=True)
df_train.head()

In [ ]:
df_train.tail()

Generamos el desplazamiento de nuestro nuestro target en el set de testeo (lag=1):

In [ ]:
df_test["valueShift1"] = df_test.value.shift()

# Podemos reemplazar el primer nan con el último valor del set de entrenamiento:
df_test.iloc[0,17] = df_train.iloc[-1,0]
df_test.head()

Vamos a hacer un scatterplot entre las observaciones y su lag de un período:

In [ ]:
df_train.plot(kind= "scatter", y = "value", x = "valueShift1", s = 50);

Ploteamos la diferencia entre nuestro target y el lag de un período:

In [ ]:
df_train["valueDiff"] = df_train.value - df_train.valueShift1
df_train.valueDiff.plot();

Como la perturbación aleatoria tiene media igual a cero, la predicción del Random Walk va a ser el valor del lag de un período:

$$ Y_t = Y_{t-1} $$

In [ ]:
df_train["RandomWalk"] = df_train.valueShift1
df_train.plot(kind="line", y = ["value", "RandomWalk"]);

La predicción sobre el set de testeo es simplemente la última observación registrada en el set de entrenamiento. 

In [ ]:
df_test["RandomWalk"] = pd.Series(df_train["value"][-1], index=df_test.index)

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind="line", y = ["value", "RandomWalk"]);

In [ ]:
# Calculamos el RMSE y almacenamos los resultados
df_Results.loc[1, "Model"] = "Random Walk"
df_Results.loc[1, "RMSE"] = RMSE(df_test.RandomWalk, df_test.value)
df_Results

Vemos que tomar la última observación del set de entranamiento mejoró significaticamente nuestras predicciones en el set de testeo respecto a tomar la media de todo el dataset.

<a id="section_tendencia_lineal"></a> 
#### 3.4- Tendencia Lineal:
[volver a TOC](#section_toc)

Continuaremos trabajando con la tendencia lineal entre value y time, fiteando nuestro modelo de regresión lineal entre "Value" y "timeIndex". En esta práctica vamos a usar la API formula de statsmodels, pero podríamos usar cualquier otra librería para realizar la regresión. 

In [ ]:
model_linear = smf.ols('value ~ timeIndex', data = df_train).fit()

El primer argumento corresponde a la forma funcional de nuestra estimación. [Más detalles](http://www.statsmodels.org/dev/examples/notebooks/generated/formulas.html).

In [ ]:
model_linear.summary()

Observamos los resultados de la estimación del modelo. Vemos que la tendencia lineal es una variable significativa con un p-valor prácticamente igual a cero y que el R2 es igual a 0,853.


Veamos las predicciones del modelo sobre el set de entrenamiento:

In [ ]:
df_train["LinearTrend"] = model_linear.predict(df_train.timeIndex)

Ploteamos los resultados de la predicción:

In [ ]:
df_train.plot(kind = "line", y = ["value","LinearTrend"]);

Veamos las predicciones del modelo sobre el set de testeo:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test["LinearTrend"] = model_linear.predict(df_test.timeIndex)

Ploteamos los resultados de la predicción:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", y = ["value","LinearTrend"]);

Calculamos el error y añadimos estos datos a nuestro DF de resultados para comparar con el modelo anterior:

In [ ]:
df_Results.loc[2, "Model"] = "LinearTrend"
df_Results.loc[2, "RMSE"] = RMSE(df_test.LinearTrend, df_test.value)
df_Results

Vemos que en este caso la tendencia lineal no logra reducir el RMSE en el set de testeo. 

<a id="section_tendencia_cuadratica"></a> 
#### 3.5- Tendencia Cuadrática:
[volver a TOC](#section_toc)

Continuaremos trabajando con la tendencia cuadrática entre value y time:

In [ ]:
# Ahora fiteamos nuestro modelo de regresión lineal entre value y timeIndex + timeIndex_sq

model_quadratic = smf.ols('value ~ timeIndex + timeIndex_sq', data = df_train).fit()

In [ ]:
# Observamos el summary del modelo:

# POR FAVOR COMPLETÁ CON TU CÓDIGO:
model_quadratic.summary()

Vemos que la componente cuadrática es significativa, ya que también tiene un p-valor cercano a cero y el R2 mejora a 0,886. En este caso, en realidad es más correcto comparar R2 ajustados ya que estamos comparando modelos con diferentes cantidad de features. Vemos entonces que el R2-ajustado mejora de 0,852 en el modelo lineal a 0,884. Podemos ver además que tanto el AIC como el BIC se redujeron, corroborando una mejor bondad de ajuste. 

Veamos las predicciones del modelo sobre el set de entrenamiento:

In [ ]:
df_train["QuadraticTrend"] = model_quadratic.predict(df_train[["timeIndex",\
                                                      "timeIndex_sq"]])

Ploteamos los resultados de la predicción:

In [ ]:
df_train.plot(kind = "line", y = ["value","QuadraticTrend"]);

Veamos las predicciones del modelo sobre el set de testeo:

In [ ]:
df_test["QuadraticTrend"] = model_quadratic.predict(df_test[["timeIndex",\
                                                      "timeIndex_sq"]])

Ploteamos los resultados de la predicción:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", y = ["value", "QuadraticTrend"]);

Agregamos las predicciones al DataFrame de testeo:

In [ ]:
# Añadimos estos datos a nuestro DF de resultados para comparar con el modelo anterior
df_Results.loc[3, "Model"] = "QuadraticTrend"
df_Results.loc[3, "RMSE"] = RMSE(df_test.QuadraticTrend, df_test.value)
df_Results

Ahora sí, vemos que trabajar con un trend cuadrático mejora el RMSE, llevándolo a 3,62.

<a id="section_tendencia_transformacion_log"></a> 
#### 3.6- Tendencia con transformación logarítmica:
[volver a TOC](#section_toc)

Analizando los datos, vemos que la varianza de la serie aumenta con el paso del tiempo. 

Veamos si una transformación logarítimica con ayuda a estabilizar la varianza:

In [ ]:
df_train['log_value'] = np.log(df_train['value'])
df_test['log_value'] = np.log(df_test['value'])

In [ ]:
plot_df(df_train, x=df_train.index, y=df_train['log_value'],\
    title='Log de ventas mensuales de medicamentos para diabetes en Australia, 1992 a 2008')

Ahora fiteamos nuestro modelo de regresión lineal entre log_value y timeIndex:

In [ ]:
model_log = smf.ols('log_value ~ timeIndex ', data = df_train).fit()

Veamos el summary del modelo:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
model_log.summary()

Vemos que trabajar con la transformación logarítmica mejora la bondad de ajuste del modelo ya que el R2 ajustado es de 0,917.  Podemos ver además que tanto el AIC como el BIC volvieron a reducirse, indicando una mejor bondad de ajuste. 

Veamos las predicciones:

In [ ]:
df_train['model_log'] = model_log.predict(df_train[["timeIndex"]])
df_test['model_log'] = model_log.predict(df_test[["timeIndex"]])

Almacenamos las predicciones del modelo en el set de entrenamiento y testeo luego de haber realizado back-transformation:

In [ ]:
df_train['back_model_log'] = np.exp(df_train['model_log'])
df_test['back_model_log'] = np.exp(df_test['model_log'])

In [ ]:
df_test.head()

Ploteamos los resultados de la predicción en el set de entrenamiento **sin** back-transformation:


In [ ]:
df_train.plot(kind = "line", y = ['log_value', 'model_log']);

Ploteamos los resultados de la predicción en el set de entrenamiento **con** back-transformation:

In [ ]:
df_train.plot(kind = "line", y = ['value', 'back_model_log']);

Ploteamos los resultados de la predicción en el set de testeo **sin** back-transformation:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", y = ['log_value', 'model_log']);

Ploteamos los resultados de la predicción en el set de testeo **con** back-transformation:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", y = ['value', 'back_model_log']);

In [ ]:
# Añadimos estos datos a nuestro DF de resultados para comparar con el modelo anterior:
df_Results.loc[4, "Model"] = "Transf Log"
df_Results.loc[4, "RMSE"] = RMSE(df_test['back_model_log'], df_test['value'])
df_Results

Vemos una mejoría en el RMSE con el modelo logarítmico. 

<a id="section_tendencia_transformacion_log_estacionalidad"></a> 
#### 3.7- Tendencia con transformación logarítmica + estacionalidad mensual:
[volver a TOC](#section_toc)

Ahora fiteamos nuestro modelo de regresión lineal entre log_value y timeIndex más las dummies de mes:

In [ ]:
model_log_est = smf.ols('log_value ~ timeIndex + Aug + Dec + Feb + Jan + Jul + Jun + Mar + May + Nov + Oct + Sep',\
                          data = df_train).fit()

In [ ]:
model_log_est.summary()

<div id="caja1" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_seguir_pensando.png" style="align:left"/> </div>
  <div style="float:left;width: 85%;"><label>¿Todos los meses son significativos?<br/> 

Vemos que al agregar las dummies mensuales, la bondad del modelo mejora sustancialmente. El R2 ajustado pasa de 0,917 a 0.987. Por otro lado, vemos que los coeficientes de todos los meses son significativos, con excepción del mes de marzo, cuyo p-valor es de 0,332. Podríamos volver a entrenar el modelo sin el mes de marzo para ver si mejora. En este caso también, los criterios AIC y BIC se redujeron, confirmando la mejoría en la bondad de ajuste indicada por el R2 ajustado.


Calculamos las predicciones:

In [ ]:
df_train['model_log_est'] = model_log_est.predict(df_train[["timeIndex", \
                                              "Aug", "Dec", "Feb", "Jan",\
                                               "Jul", "Jun", "Mar", "May",\
                                                "Nov", "Oct", "Sep"]])


df_test['model_log_est'] = model_log_est.predict(df_test[["timeIndex", \
                                              "Aug", "Dec", "Feb", "Jan",\
                                               "Jul", "Jun", "Mar", "May",\
                                                "Nov", "Oct", "Sep"]])

Almacenamos las predicciones del modelo en el set de entrenamiento y testeo luego de haber realizado back-transformation:

In [ ]:
df_train['back_model_log_est'] = np.exp(df_train['model_log_est'])
df_test['back_model_log_est'] = np.exp(df_test['model_log_est'])

In [ ]:
df_test.head()

Ploteamos los resultados de la predicción en el set de entrenamiento **sin** back-transformation:

In [ ]:
df_train.plot(kind = "line", y = ['log_value', 'model_log_est']);

Ploteamos los resultados de la predicción en el set de entrenamiento **con** back-transformation:

In [ ]:
df_train.plot(kind = "line", y = ['value', 'back_model_log_est']);

Ploteamos los resultados de la predicción en el set de testeo **sin** back-transformation:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", y = ['log_value', 'model_log_est']);

Ploteamos los resultados de la predicción en el set de testeo **con** back-transformation:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", y = ['value', 'back_model_log_est']);

In [ ]:
df_Results.loc[5, "Model"] = "Transf Log + est"
df_Results.loc[5, "RMSE"] = RMSE(df_test['back_model_log_est'], df_test['value'])
df_Results

Vemos que al modelar la estacionalidad mensual agregando las variables dummy, nuestro modelos mejora sustancialmente, reduciéndose el RMSE de 3,25 a 2,05.

<a id="section_exponential_smoothing"></a> 
#### 3.8- Single Exponential Smoothing:
[volver a TOC](#section_toc)

El **suavizamiento exponencial simple** otorga más importancia a la información reciente y menos a la pasada. En su forma más simple podemos expresarlo como:

$$ \widehat{y_t} =\alpha . y_{t-1}  + (1 - \alpha ) . \widehat{y}_{t -1} $$

Aquí el modelo es un promedio ponderado entre el valor actual y el valor anterior del modelo.
$\alpha$  se conoce como smoothing factor.

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

model_exp_smoothing = SimpleExpSmoothing(df_train.value).fit(smoothing_level=0.3,\
                                                            optimized=False)

In [ ]:
df_train.plot(kind = "line", y = "value")
model_exp_smoothing.fittedvalues.plot();

###### Time Series Cross Validation 

Antes de comenzar a construir un modelo, veamos primero cómo optimizar los hiperparámetros del modelo automáticamente.

¿Cómo podemos hacer para realizar validación cruzada con series de tiempo? Las series de tiempo tienen una estructura temporal y no se pueden mezclar los valores aleatoriamente en una partición mientras se conserva esta estructura. Con la aleatorización, se perderán todas las dependencias temporales entre las observaciones. Es por esto que tendremos que usar un enfoque más complicado para optimizar los parámetros del modelo. Utilizaremos es "cross validation on a rolling basis".

La idea es bastante simple: entrenamos nuestro modelo en un pequeño segmento de la serie de tiempo desde el principio hasta algunas t, hacemos predicciones para los siguientes t + n pasos y calculamos un error. Luego, expandimos nuestra muestra de entrenamiento a valor t + n, hacemos predicciones desde t + n hasta t + 2 ∗ n, y continuamos moviendo nuestro segmento de prueba de la serie de tiempo hasta que alcanzamos la última observación disponible. Como resultado, tenemos folds como n cabrá entre la muestra de entrenamiento inicial y la última observación

<img src="img/ts_validation.png" alt="drawing" width="600"/>

Aplicamos un split entre train y test para series de tiempo:

In [ ]:
from sklearn.model_selection import TimeSeriesSplit 

tscv = TimeSeriesSplit(n_splits=5)

Con fines ilustrativos, vamos a hacer un print de los elementos del set de entrenamiento y validación para cada fold:

In [ ]:
for train_index, val_index in tscv.split(df_train):
    print("TRAIN:", train_index, "VAL:", val_index)

Definimos una función que aplica time series cross validation para el modelo single exponential smoothing:

In [ ]:
from sklearn.metrics import mean_squared_error

def timeseriesCVscore_exp_smoot(alpha, series):
    """
        Returns error on CV  
        
        params - vector of parameters for optimization
        series - dataset with timeseries
        slen - season length for Holt-Winters model
    """
    # creamos un array de errores:
    errors = []
    
    values = series.values
    
    # instanciamos el objeto que realiza el tscv:
    tscv = TimeSeriesSplit(n_splits=5) 
    
    # Aplicamos cross validation:

    for train, test in tscv.split(values):
    
        model = SimpleExpSmoothing(values[train]).fit(smoothing_level=alpha,\
                                                             optimized=False)
        
        predictions = model.forecast(len(test))
        actual = values[test]
    
        error = mean_squared_error(predictions, actual)
        errors.append(error)
        
    return np.mean(np.array(errors))

In [ ]:
alphas = [0.001, 0.01, 0.1, 0.2, 0.3, 0.35, 0.4, 0.5, 0.7]
errors = []

for alpha in alphas:
    error = timeseriesCVscore_exp_smoot(alpha, df_train.value)
    errors.append(error)

print('Alpha óptimo:', alphas[np.argmin(errors)])

In [ ]:
# Entrenamos el modelo optimizado:

model_exp_smoothing = SimpleExpSmoothing(df_train.value).fit(smoothing_level=alphas[np.argmin(errors)],\
                                                             optimized=False)

In [ ]:
df_test["Simple_Smoothing"] = model_exp_smoothing.forecast(12)
df_test.head()

In [ ]:
model_simp_smo_RMSE = RMSE(df_test["Simple_Smoothing"], df_test.value)
model_simp_smo_RMSE

In [ ]:
df_test.plot(kind="line", y = ["value", "Simple_Smoothing"]);

In [ ]:
# Calculamos el RMSE y almacenamos los resultados
df_Results.loc[6, "Model"] = "Simple Smoothing"
df_Results.loc[6, "RMSE"] = RMSE(df_test["Simple_Smoothing"], df_test.value)
df_Results

In [ ]:
# Ploteamos las mejores estimaciones 
df_test.plot(kind = "line", y = ["value", "RandomWalk", "back_model_log_est",\
                                                 "back_model_log"]);

<a id="section_modelos_ciclos"></a> 
###  4- Modelos de Ciclos para Series de Tiempo
[volver a TOC](#section_toc)

Vamos ahora a modelar la **componente cíclica** de las series de tiempo. 

Recordemos que podíamos descomponer a una serie de tiempo de la siguiente manera:

$$ y_t = T_t + S_t + C_t + \varepsilon_t. $$

Por **ciclos** se entiende cualquier tipo de dinámica no capturada por la tendencia o la estacionalidad.

Estamos ante la presencia de ciclos cuando observamos algún tipo de dinámica mediante la cual el presente está vinculado al pasado. No tiene por qué ser un ciclo rígido.

Los ciclos son más complejos de analizar que la tendencia y la estacionalidad.

<a id="section_series_estacionarias"></a> 
#### 4.1- Series Estacionarias
[volver a TOC](#section_toc)

Solamente podemos trabajar con ciclos si tenemos series estacionarias. Comencemos por definir qué es una **serie estacionaria en sentido débil** o **estacionaria en la covarianza**:

$$ 1)\hspace{1cm}E(y_t) = \mu,  \forall t. $$
$$ 2)\hspace{1cm}Var(y_t) = \gamma < \infty,  \forall t. $$
$$ 3)\hspace{1cm}Cov(y_t, y_{t-k}) = \gamma_k < \infty,  \forall t,k. $$



1) Nos indica que la esperanza de $y_t$ es constante a lo largo del tiempo, es decir que no depende de t.

2) La varianza debe ser un valor constante y finito para todo t.

3) La autocovarianza es la covarianza de la variable contra sí misma en otros momentos del tiempo. Debe depender solamente de la traslación k y no de t.

Ejemplo de serie estacionaria:

<img src="img/Picture10.png" width="750">

Si bien la mayoría de las series no son estacionarias, es posible trabajar con modelos que tratan con los componentes no estacionarios (como el trend y la estacionalidad) de modo tal de que el componente cíclico remanente sea estacionario.

En el ejemplo de abajo, vemos que el residuo de la serie, una vez modelada la tendencia, es una serie estacionaria:

<img src="img/Picture7.png" width="450">

Alternativamente, podemos realizar transformaciones a nuestra serie para hacerla estacionaria. Podemos por ejemplo aplicar **diferenciaciones** de diferente orden para estabilizar la media o aplicar **transformaciones Box-Cox** para estabilizar la varianza. 

###### Test de Dickey-Fuller

Para evaluar la estacionariedad de una serie utilizamos el test de Dickey-Fuller (DF), o el test de Dickey-Fuller aumentado (ADF). 

•Si partimos del modelo $X_t = rX_{t-1} + u_t$

•Donde -1≤ r ≤ 1

•Si r=1 estamos en el caso de raíz unitaria y la serie no es estacioria (el Random Walk no es una series estacionaria).

•El test de Dickey-Fuller (DF) se basa en la siguiente regresión:
    $DX_t = dX_{t-1} + u_{t}$ , donde $d = (r-1)$

•La hipótesis nula $H_0$ es que los datos son no estacionarios 

•$H_0$ se rechaza si el estimador de d es negativo y significativamente diferente de cero.

En el DF se supone que el $u_t$ no está autocorrelacionado. El ADF contempla esta posibilidad adicionando al DF los valores rezagados de la variable dependiente. La $H_0$ es idéntica.

Vamos a calcular los resíduos del mejor modelo que tenemos hasta ahora, el modelo con transformación logarítmica y dummies estacionales.

Probamos primero si los residuos del modelo en la serie original son estacionarios:

In [ ]:
res_model = df_train['value'] - df_train['back_model_log_est']

plt.plot(df_train.timeIndex, res_model, '-');

Aplicamos el test de Dickey-Fuller al resíduo:

In [ ]:
# Testeamos la estacionariedad de los residuos:

from statsmodels.tsa.stattools import adfuller

result = adfuller(res_model)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in  result[4].items():
    print('Valor crítico %s: %.2f' % (key,value))

No podemos rechazar la $H_0$ con un nivel de significación del 5%. Probamos ahora con los residuos antes de realizar back transform:

In [ ]:
res_log_est = df_train['log_value'] - df_train['model_log_est']

plt.plot(df_train.timeIndex, res_log_est, '-');

In [ ]:
# Testeamos la estacionariedad de los residuos:

from statsmodels.tsa.stattools import adfuller

result = adfuller(res_log_est)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
for key, value in  result[4].items():
    print('Valor crítico %s: %.2f' % (key,value))

Podemos rechazar la $H_0$. Avanzamos con el modelado de los ciclos de los residuos del modelo:

<a id="section_modelo_arima"></a> 
#### 4.2- Modelo ARIMA
[volver a TOC](#section_toc)

El modelo ARIMA sin estacionalidad surge por la combinación de:
* un modelo Auto-Regresivo AR(p), basado en la diferenciación o rezago de la serie y
* un modelo de media móvil MA(q).

Su denominación es producto de las siglas en inglés: **AutoRegressive Integrated Moving Average** model.

Veamos un mayor detalle de los dos modelos que componen ARIMA:

##### Modelos Auto-Regresivos - AR (p)

En un modelo de autorregresión, pronosticamos la variable de interés usando una combinación lineal de valores pasados de la variable en cuestión. El término autoregresión indica que es una regresión de la variable contra sí misma. Por lo tanto, un modelo de **orden autorregresivo (p)** se puede escribir como:

$$ y_t = c + \phi_1 y_{t-1} + \phi_2 y_{t-2} + \phi_3 y_{t-3} + .. \\ $$
El Random Walk es un modelo AR (1) con $$ \phi_1 = 1,  c = 0 \\ $$ El Random Walk with drift (deriva) $$ \phi_1 = 1,  c \ != 0 \\ $$

Normalmente restringimos los modelos autorregresivos a datos estacionarios; también pueden ser necesarias algunas condiciones sobre los valores de los parámetros. Para un modelo AR (1), la condición de estacionariedad se determina como: $$ |\phi_1| < 1 $$.

##### Modelo de media móvil - MA (q)

En lugar de utilizar valores pasados de la variable a pronosticar mediante una regresión, un modelo de media móvil usa **shocks inobservables presentes y pasados** en un modelo similar a una regresión.

$$ y_t = c + e_t + \theta_1 e_{t-1} + \theta_2 e_{t-2} + ... + \theta_q e_{t-q} \\ $$

donde **$e$ es ruido blanco**. Nos referimos a esto como un modelo **MA (q)**, donde $q$ representa la **cantidad de rezagos** considerados en el modelo. Por supuesto, no observamos los valores de $ {e_t} $, por lo que no es realmente regresión en el sentido habitual.

Veamos que cada valor de $ {y_t} $ se puede considerar como una media móvil ponderada de los últimos errores de pronóstico. Sin embargo, los modelos basados en la media móvil no deben confundirse con el suavizado del promedio móvil. Se usa un modelo de promedio móvil para pronosticar valores futuros, mientras que el suavizado promedio móvil se usa para estimar el ciclo de tendencia de valores pasados.

##### Modelo ARIMA 

El modelo completo se puede escribir como:

* **Número de términos AR (autorregresivos) (p)**: los términos AR son solo rezagos de la variable dependiente. Por ejemplo, si *p* es 5, los predictores para  $ {y_t} $  serán  $ y_{t-1} $, $ y_{t-2} $  ... y  $ y_{t-5} $ .

* **Número de términos MA (promedio móvil) (q)**: los términos MA son errores de pronóstico rezagados en la ecuación de predicción. Por ejemplo, si *q* es 5, los predictores para $ {y_t} $ serán $ {e_t} $, $ e_{t-1} $, $ e_{t-2} $ ... $ e_{t-5} $ donde $ {e_t} $ es la diferencia entre el valor estimado y el valor instantáneo y real.

* **Número de diferencias (d)**: son las diferencias no estacionales que necesitamos para la estacionariedad. Entonces, o podemos pasar la variable diferenciada y poner d* = 0 o pasar la variable original y poner d* = d. Ambos generarán los mismos resultados.

Una preocupación importante aquí es cómo determinar el valor de *p* y *q*. Usamos dos vías para determinar estos parámetros, veamos:

* **Función de autocorrelación (ACF)**: es una medida de la correlación entre la serie y su rezago, también se conoce como la función de autocorrelación total. La ACF en el rezago *k*, denotada por $ \rho_k $, se define como:

$$ \rho_k = \frac {\gamma_k} {\gamma_0} $$

* **Función de autocorrelación parcial (PACF)**: mide la correlación entre la serie y su rezago después de eliminar las variaciones ya explicadas por las comparaciones intermedias. Por ejemplo, en el desfasaje 5, comprueba la correlación tras eliminar los efectos ya explicados por los rezagos 1 a 4.

En el modelo MA, el ruido (shock) se desvanece rápidamente con el tiempo. En sentido opuesto, el modelo AR tiene un efecto muy duradero del shock. De ahí que, mientras el primero se enfoca en períodos cortos de tiempo, el segundo prioriza el más largo plazo.

Importamos plots ACF y PACF:

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

Calculamos la ACF para res_log con 20 rezagos:

In [ ]:
lag_acf = acf(res_log_est, nlags = 20)
lag_acf

Almacenamos la ACF en una Serie de Pandas y la ploteamos:

In [ ]:
ACF = pd.Series(lag_acf)
ACF.plot(kind = "bar");

Calculamos la PACF para res_log con 20 rezagos. Usamos mínimos cuadrádos ordinarios para estimar los parámetros:

In [ ]:
lag_pacf = pacf(res_log_est, nlags=20, method='ols');

Almacenamos la PACF en una Serie de Pandas y la ploteamos:

In [ ]:
PACF = pd.Series(lag_pacf)
PACF.plot(kind = "bar");

Vamos a definir una función que va a plotear una serie y nos va a dar información sobre su estacionariedad y sobre sus ACF y PACF:

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    """ 
        Plotea la serie de tiempo, el ACF y PACF y el test de Dickey–Fuller
        
        y - serie de tiempo
        lags - cuántos lags incluir para el cálculo de la ACF y PACF
        
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        
        # definimos ejes
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        
        # obtengo el p-value con h0: raiz unitaria presente
        p_value = sm.tsa.stattools.adfuller(y)[1]
        
        ts_ax.set_title('Análisis de la Serie de Tiempo\n Dickey-Fuller: p={0:.5f}'\
                        .format(p_value))
        
        # plot de autocorrelacion
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        # plot de autocorrelacion parcial
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
# Corremos la función con nuestra serie res_log:

tsplot(res_log_est, lags=36)

Vemos que el gráfico nos sugiere valores para $p$ y para $q$. 

En este caso tenenemos: $p = 3$ y $q = 3$:

In [ ]:
# Importamos el modelo ARIMA de statsmodels

from statsmodels.tsa.arima_model import ARIMA

Instancio el modelo con parámetros ($p=3$, $d=0$, $q=3$) según el análisis de ACF y PACF.

En este caso $d=0$ porque trabajamos directamente con una serie estacionaria. 

In [ ]:
model_ARIMA = ARIMA(res_log_est, order=(3,0,3))

# Estimo el modelo:
results_ARIMA = model_ARIMA.fit()
results_ARIMA.fittedvalues.head()

Hacemos un print de los resultados:

In [ ]:
print(results_ARIMA.summary())

Vemos que el coeficiente ma.L1 no es significativo, pero como los rezagos de orden mayor sí lo son, dejamos los 3 coeficientes en el modelo. 

Ploteamos la serie res_log_est y las estimaciones del modelo:

In [ ]:
plt.figure(figsize=(7,3.5))
res_log_est.plot()
results_ARIMA.fittedvalues.plot();

Aplicamos el método plot_predict():

In [ ]:
results_ARIMA.plot_predict(end=len(df['value']));

Analizamos los residuos del modelo ARIMA:

In [ ]:
res_ARIMA =  results_ARIMA.fittedvalues - res_log_est

In [ ]:
tsplot(res_ARIMA, lags=36)

No parecería haber más estructura en los resíduos que podamos capturar. Nuestro resíduo ahora es ruido blanco.

Con el método forecast() podemos hacer predicciones out-of-sample:

In [ ]:
predictions_ARIMA, se, conf = results_ARIMA.forecast(len(df_test['value']), alpha=0.05)

Incorporamos el efecto cíclico a nuestro modelo, sin realizar back-transformation:

In [ ]:
df_train['log_model_ARIMA'] = df_train['model_log_est'] + results_ARIMA.fittedvalues

df_test['log_model_ARIMA'] = df_test['model_log_est'] + predictions_ARIMA

In [ ]:
df_train.plot(kind = "line", y = ['log_value', 'log_model_ARIMA']);

In [ ]:
df_test.plot(kind = "line", y = ['log_value', 'log_model_ARIMA']);

Realizamos back-transformation al modelo logarítmico:

In [ ]:
df_train['back_log_model_ARIMA'] = np.exp(df_train['log_model_ARIMA'])

df_test['back_log_model_ARIMA'] = np.exp(df_test['log_model_ARIMA'])

Ploteamos los resultados de la predicción con back-transformation en el set de train:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_train.plot(kind = "line", y = ['value', 'back_log_model_ARIMA']);

Ploteamos los resultados de la predicción con back-transformation en el set de test:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_test.plot(kind = "line", y = ['value', 'back_log_model_ARIMA']);

Calculamos el RMSE y lo almacenamos en nuestro DataFrame:

In [ ]:
df_Results.loc[7, "Model"] = "Log Model + est + ARIMA"
df_Results.loc[7, "RMSE"] = RMSE(df_test['back_log_model_ARIMA'], df_test['value'])
df_Results

Vemos que el modelo que combina la tendencia, la estacionalidad y el ciclo es el que performa mejor de todos. 

<div id="caja7" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/en_resumen.png" style="align:left"/> </div>
  <br>
  <br>  
  <div style="float:left;width: 85%;"><label><b>En conclusión...</b></label></div>
</div>
<br>

Podemos descomponer una serie de tiempo en:

- **Tendencia**: componente “permanente”, el efecto persiste en el largo plazo. Se puede interpretar como lo que motiva el cambio a largo plazo de la media.
- **Estacionalidad**: esta componente aporta movimientos periódicos a la serie.
- **Ciclos**: se entiende por cualquier tipo de dinámica no capturada por la tendencia o la estacionalidad. Estamos ante la presencia de ciclos cuando observamos algún tipo de dinámica mediante la cual el presente está vinculado al pasado. No tiene por qué ser un ciclo rígido.
- **Componente aleatoria**: son shocks que no presentan un efecto duradero, ya que las suponemos i.i.d. con media 0 y varianza constante.

De manera aditiva, podemos decir entonces que: 

$$ y_t = T_t + S_t + C_t + \varepsilon_t. $$

- Para modelar la tendencia y la estacionalidad podemos usar dummies de tiempo y estacionales. 

- Para modelar los ciclos usamos modelos ARIMA. Para modelar modelos ARIMA necesitamos que las series sean estacionarias. Las condiciones de estacionariedad son: 

$$ 1)\hspace{1cm}E(y_t) = \mu,  \forall t. $$
$$ 2)\hspace{1cm}Var(y_t) = \gamma < \infty,  \forall t. $$
$$ 3)\hspace{1cm}Cov(y_t, y_{t-k}) = \gamma_k < \infty,  \forall t,k. $$


Podríamos directamente comenzar a trabajar modelando los ciclos, sin pasar por la tendencia y la estacionalidad, pero en ese caso antes deberíamos asegurarnos de que la serie es estacionaria o realizar transformaciones para que lo sea, como por ejemplo diferenciar o aplicar transformaciones de Box-Cox.

###### Recordar que nuestro objetivo principal es que los resíduos de la serie sean ruido blanco, es decir que no quede más estructura en la serie que podamos extraer.

<div id="caja7" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/para_saber_mas.png" style="align:left"/> </div>
  <br>
  <br>  
  <div style="float:left;width: 85%;"><label><b>Para saber más:</b></label></div>
</div>
<br>

Pueden explorar dos recursos muy interesantes para trabajar con series de tiempo: Prophet y tsfresh:

[Prophet](https://github.com/facebook/prophet)

[tsfresh](https://tsfresh.readthedocs.io/en/latest/)

[volver a TOC](#section_toc)